In [1]:
import pickle

# Load the pickle file
with open('Vectordatabase/financial_chunks_metadata_sentences.pkl', 'rb') as file:
    text_chunks = pickle.load(file)

In [2]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Initialize SentenceTransformer model
text_embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Load the existing FAISS index
index = faiss.read_index('Vectordatabase/financial_docs_text_index_sentences.faiss')

def retrieve_similar(query, k=1):
    # Get query embedding
    query_embedding = text_embedder.encode([query])
    # Search in FAISS index
    distances, indices = index.search(query_embedding.astype(np.float32), k)
    return indices, distances

c:\Users\David\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2:1b")

def generate_response(query):
    indices, distances = retrieve_similar(query)
    context = "\n".join([text_chunks[idx]['raw_text'] for _, idx in enumerate(indices[0])])
    # Prepare the input for the T5 model
    input_text = f"question: {query} context: {context}. Let me know if the context was useful."



    response = llm.invoke(input_text)

    return context, response

In [3]:
import pandas as pd

# Function to convert text file to DataFrame
def text_to_dataframe(file_path):
    questions = []
    answers = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read().strip()
        entries = content.split('\n\n')  # Split on double newline for each Q&A
        
        for entry in entries:
            if entry.strip():  # Ensure the entry is not empty
                lines = entry.split('\n')
                if len(lines) >= 2:  # Ensure there are at least 2 lines (Q and A)
                    question = lines[0].replace('Q: ', '').strip()
                    answer = lines[1].replace('A: ', '').strip()
                    questions.append(question)
                    answers.append(answer)

    # Create a DataFrame
    df = pd.DataFrame({'Question': questions, 'Answer': answers})
    return df

# Example usage
file_path = "../Akshay_Work/Answers_10k.txt"
df = text_to_dataframe(file_path)

                                             Question  \
0   What is Portillo’s business model and what typ...   
1   How many restaurants did Portillo’s operate as...   
2   What are Portillo’s growth plans for restauran...   
3   What impact did inflation have on Portillo’s f...   
4   How does Portillo’s plan to maintain profitabi...   
..                                                ...   
57  What initiatives has Portillo’s implemented to...   
58  How does Portillo’s address potential food saf...   
59  How does Portillo’s manage potential real esta...   
60  How did Portillo’s manage cash flow and profit...   
61  What are Portillo’s long-term sustainability g...   

                                               Answer  
0   Portillo’s operates as a fast-casual restauran...  
1   As of December 31, 2023, Portillo's operated 8...  
2   Portillo’s aims to grow its restaurant base by...  
3   Portillo's faced 5.5% commodity price inflatio...  
4   To manage increasing food and c

In [7]:
df['RAG_context'], df['simple_RAG'] = zip(*df['Question'].apply(generate_response))

In [8]:
df

,Question,Answer,RAG_context,simple_RAG
0,What is Portillo’s business model and what typ...,Portillo’s operates as a fast-casual restauran...,Overview of Portillo's\nPortillo’s serves icon...,"Yes, the provided context is extremely helpful..."
1,How many restaurants did Portillo’s operate as...,"As of December 31, 2023, Portillo's operated 8...",The 53rd week in fiscal 2023 included Christma...,"Yes, the context provided was useful. It outli..."
2,What are Portillo’s growth plans for restauran...,Portillo’s aims to grow its restaurant base by...,"In fiscal 2023, we opened 12 new restaurants. ...",The provided context seems useful for understa...
3,What impact did inflation have on Portillo’s f...,Portillo's faced 5.5% commodity price inflatio...,Commodity inflation was\n5.5% for the year end...,The provided context is indeed useful in under...
4,How does Portillo’s plan to maintain profitabi...,To manage increasing food and commodity prices...,Portillo's Inc. Form 10-K | 34Table of Content...,"Yes, the context provided is very useful. It p..."
...,...,...,...,...
57,What initiatives has Portillo’s implemented to...,Portillo’s launched a DEI initiative guided by...,"At Portillo’s, we embrace Diversity, Equity, I...","Yes, the context you provided was very helpful..."
58,How does Portillo’s address potential food saf...,Portillo’s has implemented a comprehensive foo...,Portillo's Inc. Form 10-K | 34Table of Content...,"Yes, the context provided is useful. It seems ..."
59,How does Portillo’s manage potential real esta...,Portillo’s manages real estate challenges by r...,Portillo's Inc. Form 10-K | 53PORTILLO'S INC.\...,The context provided is a Form 10-K report fro...
60,How did Portillo’s manage cash flow and profit...,Portillo’s achieved strong cash flow and profi...,Portillo's Inc. Form 10-K | 53PORTILLO'S INC.\...,The context provided is indeed helpful in unde...


In [9]:
df.rename(columns={'simple_RAG': 'RAG_helpful_context'}, inplace=True)

In [10]:
df.to_csv("LLM_test.csv")

In [5]:
def generate_response_with_RAG(query, context):

    input_text = f"question: {query} context: {context}. Be concise."



    response = llm.invoke(input_text)

    return response

In [9]:
import pandas as pd
df = pd.read_csv("LLM_test.csv")
df['RAG_simple'] = df.apply(lambda row: generate_response_with_RAG(row['Question'], row['RAG_context']), axis=1)

In [10]:
df.to_csv("LLM_test2.csv")

In [ ]:
df.head()


,Unnamed: 0,Question,Answer,RAG_context,RAG_helpful_context_response,RAG_simple
0,0,What is Portillo’s business model and what typ...,Portillo’s operates as a fast-casual restauran...,Overview of Portillo's\nPortillo’s serves icon...,"Yes, the provided context is extremely helpful...",Portillo's business model is centered around s...
1,1,How many restaurants did Portillo’s operate as...,"As of December 31, 2023, Portillo's operated 8...",The 53rd week in fiscal 2023 included Christma...,"Yes, the context provided was useful. It outli...","As of the end of Fiscal Year 2023 (December), ..."
2,2,What are Portillo’s growth plans for restauran...,Portillo’s aims to grow its restaurant base by...,"In fiscal 2023, we opened 12 new restaurants. ...",The provided context seems useful for understa...,Portillo's growth plans for restaurant expansi...
3,3,What impact did inflation have on Portillo’s f...,Portillo's faced 5.5% commodity price inflatio...,Commodity inflation was\n5.5% for the year end...,The provided context is indeed useful in under...,Inflation had a positive impact on Portillo's ...
4,4,How does Portillo’s plan to maintain profitabi...,To manage increasing food and commodity prices...,Portillo's Inc. Form 10-K | 34Table of Content...,"Yes, the context provided is very useful. It p...",Portillo's Inc. is exposed to commodity price ...


In [17]:
with open("LLM_test_output.txt", "w", encoding="utf-8") as file:
    for index, row in df.iterrows():
        file.write(f"Question: {row['Question']}\n")
        file.write(f"Answer: {row['Answer']}\n")
        rag_helpful = row['RAG_helpful_context_response'].replace('\n', ' ')
        file.write(f"RAG Helpful: {rag_simple}\n")
        rag_simple = row['RAG_simple'].replace('\n', ' ')
        file.write(f"RAG Simple: {rag_simple}\n\n")

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Function to get embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.numpy()


def calculate_cosine_similarity(row, compare_field):
    embedding1 = get_embedding(row[compare_field])
    embedding2 = get_embedding(row['Answer'])
    return cosine_similarity(embedding1, embedding2)[0][0]

# Apply the function to create a new column with cosine similarities
df['cosine_similarity_helpful'] = df.apply(calculate_cosine_similarity, axis=1, args=('RAG_helpful_context_response',))


In [10]:
df = pd.read_csv("LLM_test2.csv")
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Question', 'Answer', 'RAG_context',
       'RAG_helpful_context_response', 'RAG_simple'],
      dtype='object')

In [11]:
df['cosine_similarity_helpful'] = df.apply(calculate_cosine_similarity, axis=1, args=('RAG_helpful_context_response',))
df['cosine_similarity_simple'] = df.apply(calculate_cosine_similarity, axis=1, args=('RAG_simple',))



In [13]:
df['cosine_similarity_simple']

0     0.874681
1     0.787188
2     0.871293
3     0.703456
4     0.736863
        ...   
57    0.845873
58    0.726939
59    0.630078
60    0.685755
61    0.740384
Name: cosine_similarity_simple, Length: 62, dtype: float32

In [15]:
threshold=0.80
count_above_threshold = (df[f'cosine_similarity_simple'] > threshold).sum()
total_count = df.shape[0]

# Calculate the percentage
percentage_above_threshold = (count_above_threshold / total_count) * 100

print(f"Count above threshold ({threshold}): {count_above_threshold}")
print(f"Percentage above threshold ({threshold}): {percentage_above_threshold:.2f}%")


Count above threshold (0.8): 24
Percentage above threshold (0.8): 38.71%


In [16]:
threshold=0.80
count_above_threshold = (df[f'cosine_similarity_helpful'] > threshold).sum()
total_count = df.shape[0]

# Calculate the percentage
percentage_above_threshold = (count_above_threshold / total_count) * 100

print(f"Count above threshold ({threshold}): {count_above_threshold}")
print(f"Percentage above threshold ({threshold}): {percentage_above_threshold:.2f}%")


Count above threshold (0.8): 8
Percentage above threshold (0.8): 12.90%


In [17]:
df.to_csv("LLM_test_cosines.csv")